In [5]:
import numpy as np
import pyautogui
import imutils
import cv2
import pytesseract
from pynput import mouse
import time
import easyocr
import torch
#from PIL import ImageGrab
import pyscreenshot
import pyscreenshot as ImageGrab
#pytesseract.pytesseract.tesseract_cmd = r'C:\Users\adaptation\anaconda3\envs\Shaman-AI\Library\bin\tesseract.exe'
from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid
from torchvision import transforms
import matplotlib.pyplot as plt


/home/adaptationai/anaconda3/envs/shamanai/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


AttributeError: module 'PIL.TiffTags' has no attribute 'IFD'

In [43]:
class ImageProcess():
    def __init__(self):
        self.love = "ramona"
        self.reader = easyocr.Reader(['en'])
        self.model = core.Model.load('model_weights.pth', ['bird', 'hole'])

    def on_click(self, x, y, button, pressed):
        if button == mouse.Button.left:
            print('{} at {}'.format('Pressed Left Click' if pressed else 'Released Left Click', (x, y)))
        
            return False # Returning False if you need to stop the program when Left clicked.
        else:
            print('{} at {}'.format('Pressed Right Click' if pressed else 'Released Right Click', (x, y)))
            return False # Returning False if you need to stop the program when Left clicked.

    def get_position(self):
        print("Please select top corner")
        listener = mouse.Listener(on_click=self.on_click)
        listener.start()
        listener.join()
        X=pyautogui.position()
        self.topx = X[0]
        self.topy= X[1]
        time.sleep(1)
        print(self.topx)
        print(self.topy)
        print("Please select bottom corner")
        listener = mouse.Listener(on_click=self.on_click)
        listener.start()
        listener.join()
        B=pyautogui.position()
        self.bottomx = B[0]
        self.bottomy= B[1]
        print(self.bottomx)
        print(self.bottomy)
        return self.topx, self.topy, self.bottomx, self.bottomy

    def image_ocr(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        #state = cv2.cvtColor(np.array(state), cv2.COLOR_BGR2GRAY)
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        #cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(state)
        print(text)
        return text

    def easy_ocr(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        

        #state = 255 - cv2.threshold(state, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Blur and perform text extraction
        #state = cv2.GaussianBlur(state, (3,3), 0)
        #self.reader = easyocr.Reader(['en'])
        result = self.reader.readtext(state)
        return result
    
  
    
    def game_start(self,tx, ty, bx, by):
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        
        

        #state = 255 - cv2.threshold(state, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Blur and perform text extraction
        #state = cv2.GaussianBlur(state, (3,3), 0)
        #self.reader = easyocr.Reader(['en'])
        result = self.reader.readtext(state)
        print(result)
        if len(result) == 1:

            print(result[0][1])
            if result[0][1] == "RESTART":
                return True
            else:
                return False
        else:
            return False

    def game_start_2(self,tx, ty, bx, by, a):
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        #state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = ImageGrab.grab(bbox=(955, 391, 1019, 406), backend="mss", childprocess=False)
        b = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        difference = cv2.subtract(a, b)    
        result = not np.any(difference)
        if result is True:
            return True
        else:
            return False
        
    def game_start_make(self,tx, ty, bx, by):
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        np.save('games_start.npy', state)
        return state


    def get_state(self,tx, ty, bx, by):
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        state = ImageGrab.grab(bbox=(753, 186, 1227, 821), backend="mss", childprocess=False)
        #state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2GRAY)[..., np.newaxis]
        
        state= cv2.resize(state, (84, 84), interpolation=cv2.INTER_AREA)[..., np.newaxis]
        #cv2.imwrite('red.jpg', state)
        #state = torch.from_numpy(state)
        #state = state.unsqueeze(dim=0)
        return state

    def get_state_save(self,tx, ty, bx, by, count, epoc):
        count = count
        epoc = epoc
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        state = ImageGrab.grab(bbox=(871, 186, 1227, 821), backend="mss", childprocess=False)
        #state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2GRAY)[..., np.newaxis]
        
        #state= cv2.resize(state, (84, 84), interpolation=cv2.INTER_AREA)[..., np.newaxis]
        cv2.imwrite('images/flappy/state'+str(count)+str(epoc)+'.jpeg', state)
        #state = torch.from_numpy(state)
        #state = state.unsqueeze(dim=0)
        return state
    
    def get_state_test(self,tx, ty, bx, by):
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2GRAY)[..., np.newaxis]
        state2= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state2 = cv2.cvtColor(np.array(state2), cv2.COLOR_RGB2GRAY)[..., np.newaxis]
        state3= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state3 = cv2.cvtColor(np.array(state3), cv2.COLOR_RGB2GRAY)[..., np.newaxis]
        state4= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state4 = cv2.cvtColor(np.array(state4), cv2.COLOR_RGB2GRAY)[..., np.newaxis]
        #vis = np.concatenate((state, state2, state3, state4), axis=1)
        vis = np.stack((state, state2, state3, state4))
        #state= cv2.resize(state, (84, 84), interpolation=cv2.INTER_AREA)
        #state = torch.from_numpy(state)
        #state = state.unsqueeze(dim=0)
        return vis

    def get_reward(self,tx, ty, bx, by):
        topx = tx
        topy = ty
        bottomx = bx - topx
        bottomy = by - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        
        

        #state = 255 - cv2.threshold(state, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Blur and perform text extraction
        #state = cv2.GaussianBlur(state, (3,3), 0)
        #self.reader = easyocr.Reader(['en'])
        result = self.reader.readtext(state)
    
        print(result)
        if len(result) == 1:

            return int(0)
        if len(result) == 2:
                if result[1][1] == "I": 
                    return int(1)
                elif result[1][1] == "2":
                    return int(2)
                elif result[1][1] == "8":
                    return int(3)
                else:
                    return(4)
        else:
            return int(0)
        
    def image_ocr_2(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        custom_config = r'--oem 3 --psm 10'
        image = pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        gray= cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
        thresh = 255 - cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Blur and perform text extraction
        thresh = cv2.GaussianBlur(thresh, (3,3), 0)
        text = pytesseract.image_to_string(thresh, config=custom_config)
        print(text)
        return text

    def image_stream(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy 
        while True:
            try:
                state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
                state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
                cv2.imshow("Stream", state)
                if cv2.waitKey(1)& 0xFF == ord('q'):
                    break      
            except KeyboardInterrupt:
                cv2.destroyAllWindows()
                break

    def get_image(self, ):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        return state

    def check_image(self, a, b):
        difference = cv2.subtract(a, b)    
        result = not np.any(difference)
        if result is True:
            print("Pictures are the same")
        else:
            print("Pictures are different")

In [59]:
import pyscreenshot as ImageGrab

In [63]:
#image = pyscreenshot.grab()
im = ImageGrab.grab(bbox=(753, 186, 1227, 821), backend="mss", childprocess=False)
im.show()

In [44]:
from PIL import ImageGrab
import pyautogui

emark = ( 1440, 1026 )
pyautogui .click( x=1063, y=544 )

pixel = ( emark[0]-1, emark[1]-1, emark[0]+1, emark[1]+1 )
original_pixel_color = ImageGrab .grab( pixel ) .getpixel( (0,0) )
##  print( original_pixel_color )

def image_grab():
    new_pixel_color = ImageGrab .grab( pixel ) .getpixel( (0,0) )
    return new_pixel_color == original_pixel_color

def clickButton():
    ##  print('yup')
    pyautogui .click( x=1160, y=600 )

while True:
    if not image_grab():
        clickButton()
        ##  I don't know if pyautogui has a built in pause function,  
        ##  or repeat delay, but you probably want a time.sleep(1)
        ##  or similar here, so it doesn't rapid-fire autoclick
        ##  500 times in a row, once that pixel changes color.

ImportError: ImageGrab is macOS and Windows only

In [ ]:
image_process.image_ocr_2()
#a = image_process.get_image()
image_process

#np.save('test.npy', a)
#a = np.load('test.npy')
#b = image_process.get_image()
#image_process.check_image(a,b)

In [80]:
image_process = ImageProcess()
image_process.get_position()
#image_process. image_ocr()
#image_process.image_stream()
#image_process.game_start_make(1029, 486, 1030, 487)#

Please select top corner
Pressed Left Click at (955, 391)
955
391
Please select bottom corner
Pressed Left Click at (1019, 406)
1019
406


(955, 391, 1019, 406)

In [230]:
#image_process.easy_ocr()
#test= image_process.get_reward(959, 392, 1019, 446)
image_process.game_start(859, 455, 974, 479)
test

[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]


0

In [37]:
test = image_process.get_state(752, 188, 1228, 823)
test.shape

(84, 84, 1)

In [45]:

class FlappyBirdsEnv():
    def __init__(self):
        self.love = 14
        self.actions = [0,1]
        self.state = None
        self.count = 0
        self.load = True
        self.eval = False
        self.image_pro = ImageProcess()
        self.frame = []
        #self.game_start = np.load('games_start.npy')
        #self.game_start2 = pyautogui.pixel(1029, 486)
        self.game_start = ImageGrab.grab(bbox=(955, 391, 1019, 406), backend="mss", childprocess=False)
        self.game_start = cv2.cvtColor(np.array(self.game_start), cv2.COLOR_RGB2BGR)
        self.total_reward = 0
        self.epoc = 0
        #self.emark = (1029, 486)
        #self.pixel = (self.emark[0]-1, self.emark[1]-1, self.emark[0]+1, self.emark[1]+1)
        #self.original_pixel_color = ImageGrab.grab(self.pixel).getpixel((0,0))

    def controller(self):
        jack = "mose"
    
    def state_maker(self):
        frame = self.image_pro.get_state(752, 188, 1228, 823)
        #self.frame.insert(0, frame)
        #elf.frame.pop()
        #state = np.stack((self.frame[0], self.frame[1], self.frame[2], self.frame[3]))
        return frame

    def state_maker_start(self):
        frame = self.image_pro.get_state(752, 188, 1228, 823)
        f
        #self.frame.pop()
        return frame

    
        
        rame2 = self.image_pro.get_state(752, 188, 1228, 823)
        frame3 = self.image_pro.get_state(752, 188, 1228, 823)
        frame4 = self.image_pro.get_state(752, 188, 1228, 823)
        self.frame.insert(3, frame)
        self.frame.insert(2, frame2)
        self.frame.insert(1, frame3)
        self.frame.insert(0, frame4)
        state = np.stack((self.frame[0], self.frame[1], self.frame[2], self.frame[3]))
        

    def step(self, action):
        
        self.count += 1
        if action == 1:
            pyautogui.click(x=795, y= 219, button='left')
            #self.rewards = self.rewards - 0.01
        else:
            d=1
        #self.player.action(self.action)
        time.sleep(0.1)
        state = self.state_maker()
        self.image_pro.get_state_save(752, 188, 1228, 823, self.count, self.epoc)
        self.state = state
        self.rewards = 0.0015
        self.total_reward = self.total_reward + self.rewards
        
        #reward = self.reward(self.get_state(), self.bet_value)
        #self.player.balance += reward
        remainder = self.count % 1
        is_divisible = remainder == 0
        if is_divisible == True:
            done = self.check_start()

        else:
            done = False
        if done == True:
            self.rewards = -1
            self.total_reward = self.total_reward + self.rewards
            print(self.total_reward)
        #time.sleep(0.1)
        
        return state, self.rewards, done

    def check_start(self):
        check = self.image_pro.game_start_2(955, 391, 1019, 406, self.game_start)
        #check = pyautogui.pixelMatchesColor(1029, 486,(222, 216, 149))
        #self.new_pixel_color = ImageGrab.grab(pixel).getpixel((0,0))
        #check = self.new_pixel_color == self.original_pixel_color
        return check



    def reset(self):
        self.total_reward = 0
        #self.player.balance = 0
        self.count = 0
        self.epoc += 1
        for i in range(10000):
            start = self.check_start()
            if start == True:
                pyautogui.click(x=914, y=566, button='left')
                print("start")
                break
            else:
                time.sleep(3)

        #self.make_episode()
        
        #print(len(self.state))
        time.sleep(1)
        pyautogui.click(x=795, y= 219, button='left')
        time.sleep(.425)
        pyautogui.click(x=795, y= 219, button='left')
        time.sleep(.425)
        pyautogui.click(x=795, y= 219, button='left')
        time.sleep(.425)
        pyautogui.click(x=795, y= 219, button='left')
        time.sleep(.425)
        pyautogui.click(x=795, y= 219, button='left')
        time.sleep(.425)
        pyautogui.click(x=795, y= 219, button='left')
        time.sleep(.425)
        pyautogui.click(x=795, y= 219, button='left')
        
        
        
        state = self.state_maker()
        self.rewards = 0
        return state

    def render(self):
        print(self.player.balance)

    def reward(self, state):
        reward = 0
    

        return reward
    
    def done(self, count):
        if count == 10000:
            
            return True
        else:
            return False

    def start(self):
        self.something = "lala"

In [45]:
pyautogui.pixel(1029, 486)

RGB(red=222, green=216, blue=149)

In [86]:
flappy = FlappyBirdsEnv()

flappy.reset()
flappy.step(1)

[([[12, 8], [124, 8], [124, 34], [12, 34]], 'RESTART', 0.9832305775554079)]
RESTART
start


(array([[[173],
         [173],
         [173],
         ...,
         [173],
         [173],
         [173]],
 
        [[173],
         [173],
         [173],
         ...,
         [173],
         [173],
         [173]],
 
        [[173],
         [173],
         [173],
         ...,
         [173],
         [173],
         [173]],
 
        ...,
 
        [[210],
         [210],
         [210],
         ...,
         [210],
         [210],
         [210]],
 
        [[210],
         [210],
         [210],
         ...,
         [210],
         [210],
         [210]],
 
        [[210],
         [210],
         [210],
         ...,
         [210],
         [210],
         [210]]], dtype=uint8),
 0.09000000000000001,
 False)

In [266]:
remainder = 11 % 10


is_divisible = remainder == 0



print(is_divisible)

False


In [4]:
class Player():
    def __init__(self):
        self.love = "Ramona"
        self.score = 0
        self.state = []
        self.reward = []
        self.done = False

    #pyautogui.keyDown('shift')  # hold down the shift key
    #pyautogui.press('left')     # press the left arrow key
    #pyautogui.press('left')     # press the left arrow key
    #pyautogui.press('left')     # press the left arrow key
    #pyautogui.keyUp('shift')    # release the shift key

    def action(self, action):
        self.love2 = "ramona"
        
        if action == 0:
            pyautogui.press("space")
        else:   
            pyautogui.keyUp("space")

In [46]:
import numpy as np
#from .state import *
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np

#MAIN WRAPPER FOR THE ENVIRONMENT TO WORK WITH OPEN AI GYM

class Template_Gym(gym.Env):
    metadata = {
        "render.modes": ["human", "rgb_array"],
    }
    #Define Actions
    ACTION = [0,1]

    def __init__(self):
        #self.config = config
        #self.eval= self.config.eval
        self.start = 0
        self.env = FlappyBirdsEnv()
        self.viewer = None
        self.info = None
        self.reward = 0
        self.done = False
        self.state = None
        self.action_dim = 4
        self.state_dim = 32
        self.num_envs = 1
        self.num_envs_per_sub_batch = 1
        self.starter = 0
        self.discrete = True
        
        #self.shape = len(self.env.reset())
        #print(self.shape)
        #self.action_shape = len(self.env.actions)


        #self.df = df
        #self.reward_range = (0, MAX_ACCOUNT_BAL2NCE) 
        if self.discrete:
            # forward or backward in each dimension
            self.action_space = spaces.Discrete(2)
            #self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([1, 1]), dtype=np.float32)

            # observation is the x, y coordinate of the grid
            #low = np.zeros(0, dtype=int)
            #high =  np.array(1, dtype=int) - np.ones(len(self.maze_size), dtype=int)
            #aud = 3339
            #self.observation_space = spaces.Box(low=-10000, high=10000, shape=(3333,))
            self.observation_space = spaces.Box(low=0, high=255,shape=(84,84,1), dtype=np.uint8)
        else:
            # Actions of the format Buy x%, Sell x%, Hold, etc.
            self.action_space = spaces.Box(low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float32)
            #or
            #self.action_space = spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([3, 1, 1, 1]), dtype=np.float16)


            # Prices contains the OHCL values for the last five prices
            self.observation_space = spaces.Box(low=0, high=1, shape=(6, 6), dtype=np.float16)

        

        # initial condition
        #self.state = self.env.generate_number()
        self.steps_beyond_done = None

        # Simulation related variables.
        self.seed()
        #self.reset()

        # Just need to initialize the relevant attributes
        self.configure()

    def __del__(self):
        pass

    def configure(self, display=None):
        self.display = display

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        #self.state = self.env.generate_number()
        #self.env.display()
        #print(action)
        #self.placement = self.env.placement
        self.next_state, self.reward, self.done= self.env.step(action)
        #self.info = 0
        #print(self.reward)
        self.info = { 'pnl':1, 'nav':1, 'costs':1 }
        #self.next_state = self.next_state.tolist()
        
        #if self.done:
            #print("total pips")
            #print(np.sum(self.total_pips))
            #print(len(self.total_pips))
            #self.starter += 1
            #pass
        return self.next_state, self.reward, self.done, self.info

    def step_async(self, action):
        #self.state = self.env.generate_number()
        #self.env.display()
        #print(action)
        #self.placement = self.env.placement
        self.next_state, self.reward, self.done= self.env.step(action)
        #self.info = 0
        #print(self.reward)
        self.info = { 'pnl':1, 'nav':1, 'costs':1 }
        #self.next_state = self.next_state.tolist()
        
        #if self.done:
            #print("total pips")
            #print(np.sum(self.total_pips))
            #print(len(self.total_pips))
            #self.starter += 1
            #pass
        return self.next_state, self.reward, self.done, self.info

    def step_wait(self):
        #self.state = self.env.generate_number()
        #self.env.display()
        #print(action)
        #self.placement = self.env.placement
        self.next_state, self.reward, self.done= self.env.step(action)
        #self.info = 0
        #print(self.reward)
        self.info = { 'pnl':1, 'nav':1, 'costs':1 }
        #self.next_state = self.next_state.tolist()
        
        #if self.done:
            #print("total pips")
            #print(np.sum(self.total_pips))
            #print(len(self.total_pips))
            #self.starter += 1
            #pass
        return self.next_state, self.reward, self.done, self.info
    

    def reset(self):
        
        self.state = self.env.reset()
        #self.reward = np.array([reward])
        #self.state = self.state.tolist()
        #self.state = np.array([self.state])
        #self.steps_beyond_done = None
        self.done = False
        #self.done = np.array([self.done])
        return self.state

    def is_game_over(self):
        pass
        return

    def render(self, mode="human", close=False):
        #self.env.display()
        pass

        return 


test = Template_Gym()


In [197]:
import gym
import numpy as np
from gym import spaces

try:
    import cv2  # pytype:disable=import-error

    cv2.ocl.setUseOpenCL(False)
except ImportError:
    cv2 = None

from stable_baselines3.common.type_aliases import GymObs, GymStepReturn

class MaxAndSkipEnv(gym.Wrapper):
    """
    Return only every ``skip``-th frame (frameskipping)

    :param env: the environment
    :param skip: number of ``skip``-th frame
    """

    def __init__(self, env: gym.Env, skip: int = 4):
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=env.observation_space.dtype)
        self._skip = skip

    def step(self, action: int) -> GymStepReturn:
        """
        Step the environment with the given action
        Repeat action, sum reward, and max over last observations.

        :param action: the action
        :return: observation, reward, done, information
        """
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info


    def reset(self, **kwargs) -> GymObs:
        return self.env.reset(**kwargs)

In [148]:
class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0,            
                                shape=(old_shape[-1], 
                                old_shape[0], old_shape[1]),
                                dtype=np.float32)
    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)

In [149]:
class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

In [211]:
class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),old_space.high.repeat(n_steps, axis=0),dtype=dtype)
    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low,dtype=self.dtype)
        return self.observation(self.env.reset())
        
    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer

In [203]:
class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
    def observation(self, obs):
        return ProcessFrame84.process(obs)
    @staticmethod 
    def process(frame):
        if frame.size == 84 * 84 * 1:
            img = np.reshape(frame, [84, 84, 1]).astype(np.float32)
            
        elif frame.size == 250 * 160 * 3:
                img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False     
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110),interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)

In [ ]:
class VecFrameStack(gym.ObservationWrapper):
    """
    Frame stacking wrapper for vectorized environment

    :param venv: (VecEnv) the vectorized environment to wrap
    :param n_stack: (int) Number of frames to stack
    """

    def __init__(self, venv, n_stack):
        self.venv = venv
        self.n_stack = n_stack
        wrapped_obs_space = venv.observation_space
        low = np.repeat(wrapped_obs_space.low, self.n_stack, axis=-1)
        high = np.repeat(wrapped_obs_space.high, self.n_stack, axis=-1)
        self.stackedobs = np.zeros((venv.num_envs,) + low.shape, low.dtype)
        observation_space = spaces.Box(low=low, high=high, dtype=venv.observation_space.dtype)
    

    def step_wait(self):
        observations, rewards, dones, infos = self.venv.step_wait()
        last_ax_size = observations.shape[-1]
        self.stackedobs = np.roll(self.stackedobs, shift=-last_ax_size, axis=-1)
        for i, done in enumerate(dones):
            if done:
                if 'terminal_observation' in infos[i]:
                    old_terminal = infos[i]['terminal_observation']
                    new_terminal = np.concatenate(
                        (self.stackedobs[i, ..., :-last_ax_size], old_terminal), axis=-1)
                    infos[i]['terminal_observation'] = new_terminal
                else:
                    warnings.warn(
                        "VecFrameStack wrapping a VecEnv without terminal_observation info")
                self.stackedobs[i] = 0
        self.stackedobs[..., -observations.shape[-1]:] = observations
        return self.stackedobs, rewards, dones, infos


    def reset(self):
        """
        Reset all environments
        """
        obs = self.venv.reset()
        self.stackedobs[...] = 0
        self.stackedobs[..., -obs.shape[-1]:] = obs
        return self.stackedobs

In [129]:
for i in range(10):
            #
            #self.total_pips = []
            obs = test.reset()
            print(obs.shape)
            print(obs.size)
            state = None
            
            # When using VecEnv, done is a vector
            
            for i in range(100):
                # _states are only useful when using LSTM policies
                #action, state = self.model.predict(obs, state=state, mask=done, deterministic=True)
                obs, rewards , dones, _ = test.step(1)
                if dones == True:
                    break
                #actions, _states = self.model.predict(obs)
                # # here, action, rewards and dones are arrays
                 # # because we are using vectorized env
                #obs, rewards, dones, info = self.env.step(actions)
                #self.total_pips.append(self.env.player.placement)

start
(84, 84, 1)
7056
[]


KeyboardInterrupt: 

In [ ]:
import gym
import numpy as np
from gym import spaces

try:
    import cv2  # pytype:disable=import-error

    cv2.ocl.setUseOpenCL(False)
except ImportError:
    cv2 = None

from stable_baselines3.common.type_aliases import GymObs, GymStepReturn


class NoopResetEnv(gym.Wrapper):
    """
    Sample initial states by taking random number of no-ops on reset.
    No-op is assumed to be action 0.

    :param env: the environment to wrap
    :param noop_max: the maximum value of no-ops to run
    """

    def __init__(self, env: gym.Env, noop_max: int = 30):
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == "NOOP"
        
    def reset(self, **kwargs) -> np.ndarray:
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1)
        assert noops > 0
        obs = np.zeros(0)
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs



class FireResetEnv(gym.Wrapper):
    """
    Take action on reset for environments that are fixed until firing.

    :param env: the environment to wrap
    """

    def __init__(self, env: gym.Env):
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == "FIRE"
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs) -> np.ndarray:
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs



class EpisodicLifeEnv(gym.Wrapper):
    """
    Make end-of-life == end-of-episode, but only reset on true game over.
    Done by DeepMind for the DQN and co. since it helps value estimation.

    :param env: the environment to wrap
    """

    def __init__(self, env: gym.Env):
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action: int) -> GymStepReturn:
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if 0 < lives < self.lives:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info


    def reset(self, **kwargs) -> np.ndarray:
        """
        Calls the Gym environment reset, only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.

        :param kwargs: Extra keywords passed to env.reset() call
        :return: the first observation of the environment
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs



class MaxAndSkipEnv(gym.Wrapper):
    """
    Return only every ``skip``-th frame (frameskipping)

    :param env: the environment
    :param skip: number of ``skip``-th frame
    """

    def __init__(self, env: gym.Env, skip: int = 4):
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=env.observation_space.dtype)
        self._skip = skip

    def step(self, action: int) -> GymStepReturn:
        """
        Step the environment with the given action
        Repeat action, sum reward, and max over last observations.

        :param action: the action
        :return: observation, reward, done, information
        """
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info


    def reset(self, **kwargs) -> GymObs:
        return self.env.reset(**kwargs)



class ClipRewardEnv(gym.RewardWrapper):
    """
    Clips the reward to {+1, 0, -1} by its sign.

    :param env: the environment
    """

    def __init__(self, env: gym.Env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward: float) -> float:
        """
        Bin reward to {+1, 0, -1} by its sign.

        :param reward:
        :return:
        """
        return np.sign(reward)



class WarpFrame(gym.ObservationWrapper):
    """
    Convert to grayscale and warp frames to 84x84 (default)
    as done in the Nature paper and later work.

    :param env: the environment
    :param width:
    :param height:
    """

    def __init__(self, env: gym.Env, width: int = 84, height: int = 84):
        gym.ObservationWrapper.__init__(self, env)
        self.width = width
        self.height = height
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(self.height, self.width, 1), dtype=env.observation_space.dtype
        )

    def observation(self, frame: np.ndarray) -> np.ndarray:
        """
        returns the current observation from a frame

        :param frame: environment frame
        :return: the observation
        """
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]



class AtariWrapper(gym.Wrapper):
    """
    Atari 2600 preprocessings

    Specifically:

    * NoopReset: obtain initial state by taking random number of no-ops on reset.
    * Frame skipping: 4 by default
    * Max-pooling: most recent two observations
    * Termination signal when a life is lost.
    * Resize to a square image: 84x84 by default
    * Grayscale observation
    * Clip reward to {-1, 0, 1}

    :param env: gym environment
    :param noop_max: max number of no-ops
    :param frame_skip: the frequency at which the agent experiences the game.
    :param screen_size: resize Atari frame
    :param terminal_on_life_loss: if True, then step() returns done=True whenever a life is lost.
    :param clip_reward: If True (default), the reward is clip to {-1, 0, 1} depending on its sign.
    """

    def __init__(
        self,
        env: gym.Env,
        noop_max: int = 30,
        frame_skip: int = 4,
        screen_size: int = 84,
        terminal_on_life_loss: bool = True,
        clip_reward: bool = True,
    ):
        env = NoopResetEnv(env, noop_max=noop_max)
        env = MaxAndSkipEnv(env, skip=frame_skip)
        if terminal_on_life_loss:
            env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = WarpFrame(env, width=screen_size, height=screen_size)
        if clip_reward:
            env = ClipRewardEnv(env)

        super(AtariWrapper, self).__init__(env)

import warnings

import numpy as np
from gym import spaces

from stable_baselines.common.vec_env.base_vec_env import VecEnvWrapper


class VecFrameStack(VecEnvWrapper):
    """
    Frame stacking wrapper for vectorized environment

    :param venv: (VecEnv) the vectorized environment to wrap
    :param n_stack: (int) Number of frames to stack
    """

    def __init__(self, venv, n_stack):
        self.venv = venv
        self.n_stack = n_stack
        wrapped_obs_space = venv.observation_space
        low = np.repeat(wrapped_obs_space.low, self.n_stack, axis=-1)
        high = np.repeat(wrapped_obs_space.high, self.n_stack, axis=-1)
        self.stackedobs = np.zeros((venv.num_envs,) + low.shape, low.dtype)
        observation_space = spaces.Box(low=low, high=high, dtype=venv.observation_space.dtype)
        VecEnvWrapper.__init__(self, venv, observation_space=observation_space)

    def step_wait(self):
        observations, rewards, dones, infos = self.venv.step_wait()
        last_ax_size = observations.shape[-1]
        self.stackedobs = np.roll(self.stackedobs, shift=-last_ax_size, axis=-1)
        for i, done in enumerate(dones):
            if done:
                if 'terminal_observation' in infos[i]:
                    old_terminal = infos[i]['terminal_observation']
                    new_terminal = np.concatenate(
                        (self.stackedobs[i, ..., :-last_ax_size], old_terminal), axis=-1)
                    infos[i]['terminal_observation'] = new_terminal
                else:
                    warnings.warn(
                        "VecFrameStack wrapping a VecEnv without terminal_observation info")
                self.stackedobs[i] = 0
        self.stackedobs[..., -observations.shape[-1]:] = observations
        return self.stackedobs, rewards, dones, infos


    def reset(self):
        """
        Reset all environments
        """
        obs = self.venv.reset()
        self.stackedobs[...] = 0
        self.stackedobs[..., -obs.shape[-1]:] = obs
        return self.stackedobs


    def close(self):
        self.venv.close()

class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
    def observation(self, obs):
        return ProcessFrame84.process(obs)@staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
                img = np.reshape(frame, [210, 160,  3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False     
            mg = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
            resized_screen = cv2.resize(img, (84, 110),interpolation=cv2.INTER_AREA)
            x_t = resized_screen[18:102, :]
            x_t = np.reshape(x_t, [84, 84, 1])
            return x_t.astype(np.uint8)


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),old_space.high.repeat(n_steps, axis=0),dtype=dtype)
    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low,
        dtype=self.dtype)
        return self.observation(self.env.reset())
        
    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer

In [12]:
from pynput import keyboard
from pynput.keyboard import Key, Controller
moves =[False, False]

class KeyLogger():
    def __init__(self):
        self.moose = "9"
        self.listener = keyboard.Listener(on_press=self.on_press,on_release=self.on_release)
        self.listener.start()
        self.moves = [False, False, False, False, False]
        self.action = 0
    def on_press(self, key):
        try:
            print('alphanumeric key {0} pressed'.format(
                key.char))
            self.moose = key
            print(key)
            if key.char == '1':
                self.moves[1] = True
            if key.char == '0':
                self.moves[0] = True
            if key.char == '2':
                self.moves[2] = True
            if key.char == '3':
                self.moves[3] = True
            if key.char == '4':
                self.moves[4] = True
                
        
        except AttributeError:
            print('special key {0} pressed'.format(
                key))
        return 

    def on_release(self, key):
        print('{0} released'.format(
            key))
        #self.moose = ''
        moose = key
        if key.char == "0":
            self.moves[0] = False
        if key.char == "1":
            self.moves[1] = False
        if key.char == "2":
            self.moves[2] = False
        if key.char == "3":
            self.moves[3] = False
        if key.char == "4":
            self.moves[4] = False
        if key == keyboard.Key.esc:
            # Stop listener
            return False
    def actions(self):
        if self.moves[3] == True and self.moves[2] == True:
            self.action = 4  
        elif self.moves[1] == True:
            self.action = 1
        elif self.moves[2] == True:
            self.action = 2
        elif self.moves[3] == True:
            self.action = 3
        elif self.moves[3] == True:
            self.action = 3
        

        else:
            self.action = 0
        return self.action




class KeyboardController():
    def __init__(self):
        self.moose = "yeah"
        self.keyboard = Controller()
    
    def press(self, key):
        self.keyboard.press(Key.key)

    def release(self, key):
        self.keyboard.release(Key.key)

    def press_release(self, key):
        self.keyboard.press(Key.key)
        self.keyboard.release(Key.key)
    
    def typeing(self, string):
        self.keyboard.type(string)



from pynput import mouse
from pynput.mouse import Button, Controller

class MouseLogger():
    def __init__(self):
        self.test= "test"
        self.listener = mouse.Listener(on_move=self.on_move,on_click=self.on_click,on_scroll=self.on_scroll)
        self.listener.start()
        self.action = 0
        self.moves = [False, False, False, False, False]
    def on_move(self, x, y):
        print('Pointer moved to {0}'.format(
            (x, y)))

    def on_click(self, x, y, button, pressed):
        print('{0} at {1}'.format(
            'Pressed' if pressed else 'Released',
            (x, y)))
        self.moves[1] = True
        if not pressed:
            # Stop listener
            self.moves[1] = False
            return False

    def on_scroll(self, x, y, dx, dy):
        print('Scrolled {0} at {1}'.format(
            'down' if dy < 0 else 'up',
            (x, y)))

    def listener_block(self):
        with mouse.Listener(
            on_move=self.on_move,
            on_click=self.on_click,
            on_scroll=self.on_scroll) as self.listener:
            self.listener.join()
            
    def actions(self, obs):
        
        if self.moves[3] == True and self.moves[2] == True:
            self.action = 4  
        elif self.moves[1] == True:
            self.action = 1
        elif self.moves[2] == True:
            self.action = 2
        elif self.moves[3] == True:
            self.action = 3
        elif self.moves[3] == True:
            self.action = 3
        
        #a = np.array([0,1])
        else:
            self.action = 0
        r= [(0,None), (1,None)]
        #2 = (np.array(r), None)
        r2 = np.array((1, None))
        #r = ((0,None), (1,None) )
        self.response = (self.action, None)
        return self.response

class MouseController():
    def __init__(self):
        pass
        self.mouse = Controller()

    def mouse_position(self):
        # Read pointer position
        print('The current pointer position is {0}'.format(
        self.mouse.position))
        return self.mouse.position
    
    def mouse_postiion_set(self, x, y):

        # Set pointer position
        self.mouse.position = (x, y)
        print('Now we have moved it to {0}'.format(
        self.mouse.position))

    def mouse_move(self, x, y):
        # Move pointer relative to current position
        self.mouse.move(x, y)

    def mouse_press(self, right=False):
        if right == True:
            self.mouse.press(Button.right)
        else:
            self.mouse.press(Button.left)

    def mouse_release(self, right=False):
        if right == True:
            self.mouse.release(Button.right)
        else:
            self.mouse.release(Button.left)

    def mouse_double_click(self, right=False):
        # Double click; this is different from pressing and releasing
        # twice on Mac OSX
        if right == 'right' or right == True:
            self.mouse.click(Button.right, 2)
        else:
            self.mouse.click(Button.right, 2)

    def mouse_scroll(self, x, y):
        # Scroll two steps down
        self.mouse.scroll(x, y)

In [266]:
import numpy as np

import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
#from .keyboard import KeyboardController, KeyLogger
#from .mouse import MouseController, MouseLogger

class Controller_Gym(gym.Env):
    metadata = {
        "render.modes": ["human", "rgb_array"],
    }
    #Define Actions
    ACTION = [0,1]

    def __init__(self, env):
        self.controlling = True
        self.logging = True
        #self.mouse_logger = MouseLogger()
        #self.mouse_controller = MouseController()
        self.keyboard_logger = KeyLogger()
        #self.keyboard_controller = KeyboardController()
        self.env = env
        self.viewer = None
        self.info = None
        self.reward = None
        self.done = False
        self.state = None
        #self.action_dim = 3
        #self.state_dim = 109
        self.num_envs = 1
        self.num_envs_per_sub_batch = 1
        self.total_pips = []
        #self.player = self.env.player
        #self.pips = self.env.pips
        #self.starter = 0

        # forward or backward in each dimension
        #self.action_space = spaces.Discrete(3)
        self.action_space = self.env.action_space

        # observation is the x, y coordinate of the grid
        #low = np.zeros(0, dtype=int)
        #high =  np.array(1, dtype=int) - np.ones(len(self.maze_size), dtype=int)
        #self.observation_space = spaces.Box(low=-100000, high=100000, shape=(109,))
        self.observation_space = self.env.observation_space
        #print("obs")
        #print (self.observation_space)

        # initial condition
        #self.state = self.env.generate_number()
        self.steps_beyond_done = None

        # Simulation related variables.
        self.seed()
        #self.reset()

        # Just need to initialize the relevant attributes
        self.configure()

    def __del__(self):
        pass

    def configure(self, display=None):
        self.display = display

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        #self.state = self.env.generate_number()
        #self.env.display()
        #print(action)
        action = self.keyboard_logger.actions()
        #action = 1
        #self.placement = self.env.placement
        self.next_state, self.reward, self.done, info = self.env.step(action)
        #self.info = 0
        #print(self.reward)
        self.info = { 'pnl':1, 'nav':1, 'costs':1 }
        #self.next_state = self.next_state.tolist()
        #self.total_pips.append(self.pips)
        if self.done:
            pass
        return self.next_state, self.reward, self.done, info

    def reset(self):
        self.state = self.env.reset()
        #self.reward = np.array([reward])
        #self.state = self.state.tolist()
        #self.state = np.array([self.state])
        #self.steps_beyond_done = None
        self.done = False
        #self.done = np.array([self.done])
        return self.state

    def is_game_over(self):
        pass
        return

    def render(self, mode="human", close=False):
        self.env.render()
        #self.env.display()
        pass

        return 



In [271]:
from stable_baselines.gail import generate_expert_traj

from stable_baselines.gail import ExpertDataset

In [387]:
def pre_train(self, num_e=1, load="saves/m19"):
        env_id = 'default'
        num_e = 1
        log_dir = "saves"
        # Usingenv = make_env() only one expert trajectory
        # you can specify `traj_limitation=-1` for using the whole dataset
        dataset = ExpertDataset(expert_path='default2.npz',traj_limitation=1, batch_size=128)
        env = Template_Gym()
        #env = Controller_Gym(env)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, n_stack=1)
        #env = make_env()
        #model = GAIL("MlpPolicy", env=env, expert_dataset=dataset, verbose=1)
        #self.env.save_running_average("saves"+self.config.pair)
        model = PPO('CnnPolicy', env, verbose=1)
        #self.model = PPO2.load("saves/m19", self.env, policy=CustomPolicy, tensorboard_log="./default/" )
        #self.env.save_running_average("saves"+self.config.pair)
        # Pretrain the PPO2 model
        model.pretrain(dataset, n_epochs=10000)

        # As an option, you can train the RL agent
        #self.model.learn(int(100000000))

        # Test the pre-trained model
        env = model.get_env()
        #env.save_running_average("saves"+self.config.pair)
        obs = env.reset()

        reward_sum = 0.0
        for _ in range(1000000):
            action, _ = model.predict(obs)
            obs, reward, done, _ = env.step(action)
            reward_sum += reward
            #self.env.render()
            if done:
                print(reward_sum)
                reward_sum = 0.0
                obs = self.env.reset()

        self.env.close()

def gen_pre_train(game, state, num_e=1, save='default2', episodes=10):
    
    env = Template_Gym()
    env = Controller_Gym(env)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, n_stack=1)
    expert_agent = "moose"
    generate_expert_traj(expert_agent, save, env, n_episodes=episodes)

Pressed at (2247, 207)
Released at (2247, 207)


In [26]:
expert = MouseLogger()
test = expert.actions(1)
print(test)

(0, None)
Pointer moved to (2256, 237)
Pointer moved to (2257, 238)
Pointer moved to (2261, 240)
Pointer moved to (2270, 243)
Pointer moved to (2272, 244)
Pointer moved to (2287, 250)
Pointer moved to (2299, 252)
Pointer moved to (2320, 257)
Pointer moved to (2342, 263)
Pointer moved to (2366, 274)
Pointer moved to (2383, 280)
Pointer moved to (2405, 290)
Pointer moved to (2424, 303)
Pointer moved to (2437, 307)
Pointer moved to (2439, 308)
Pointer moved to (2441, 309)
Pointer moved to (2444, 309)
Pointer moved to (2446, 310)
Pointer moved to (2447, 310)
Pointer moved to (2447, 311)
Pointer moved to (2449, 311)
Pointer moved to (2450, 311)
Pointer moved to (2451, 311)
Pointer moved to (2452, 310)
Pointer moved to (2454, 309)
Pointer moved to (2456, 308)
Pointer moved to (2457, 307)
Pointer moved to (2458, 306)
Pointer moved to (2459, 305)
Pointer moved to (2460, 305)
Pointer moved to (2460, 303)
Pointer moved to (2460, 302)
Pointer moved to (2460, 301)
Pointer moved to (2460, 300)
Poin

In [14]:
from collections import deque
import numpy as np
from gym.spaces import Box
from gym import ObservationWrapper


class LazyFrames(object):
    r"""Ensures common frames are only stored once to optimize memory use.
    To further reduce the memory use, it is optionally to turn on lz4 to
    compress the observations.
    .. note::
        This object should only be converted to numpy array just before forward pass.
    Args:
        lz4_compress (bool): use lz4 to compress the frames internally
    """
    __slots__ = ("frame_shape", "dtype", "shape", "lz4_compress", "_frames")

    def __init__(self, frames, lz4_compress=False):
        self.frame_shape = tuple(frames[0].shape)
        self.shape = (len(frames),) + self.frame_shape
        self.dtype = frames[0].dtype
        if lz4_compress:
            from lz4.block import compress

            frames = [compress(frame) for frame in frames]
        self._frames = frames
        self.lz4_compress = lz4_compress

    def __array__(self, dtype=None):
        arr = self[:]
        if dtype is not None:
            return arr.astype(dtype)
        return arr

    def __len__(self):
        return self.shape[0]

    def __getitem__(self, int_or_slice):
        if isinstance(int_or_slice, int):
            return self._check_decompress(self._frames[int_or_slice])  # single frame
        return np.stack(
            [self._check_decompress(f) for f in self._frames[int_or_slice]], axis=0
        )

    def __eq__(self, other):
        return self.__array__() == other

    def _check_decompress(self, frame):
        if self.lz4_compress:
            from lz4.block import decompress

            return np.frombuffer(decompress(frame), dtype=self.dtype).reshape(
                self.frame_shape
            )
        return frame


class FrameStack(ObservationWrapper):
    r"""Observation wrapper that stacks the observations in a rolling manner.
    For example, if the number of stacks is 4, then the returned observation contains
    the most recent 4 observations. For environment 'Pendulum-v0', the original observation
    is an array with shape [3], so if we stack 4 observations, the processed observation
    has shape [4, 3].
    .. note::
        To be memory efficient, the stacked observations are wrapped by :class:`LazyFrame`.
    .. note::
        The observation space must be `Box` type. If one uses `Dict`
        as observation space, it should apply `FlattenDictWrapper` at first.
    Example::
        >>> import gym
        >>> env = gym.make('PongNoFrameskip-v0')
        >>> env = FrameStack(env, 4)
        >>> env.observation_space
        Box(4, 210, 160, 3)
    Args:
        env (Env): environment object
        num_stack (int): number of stacks
        lz4_compress (bool): use lz4 to compress the frames internally
    """

    def __init__(self, env, num_stack, lz4_compress=False):
        super(FrameStack, self).__init__(env)
        self.num_stack = num_stack
        self.lz4_compress = lz4_compress

        self.frames = deque(maxlen=num_stack)

        low = np.repeat(self.observation_space.low[np.newaxis, ...], num_stack, axis=0)
        high = np.repeat(
            self.observation_space.high[np.newaxis, ...], num_stack, axis=0
        )
        self.observation_space = Box(
            low=low, high=high, dtype=self.observation_space.dtype
        )

    def observation(self):
        assert len(self.frames) == self.num_stack, (len(self.frames), self.num_stack)
        return LazyFrames(list(self.frames), self.lz4_compress)

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.frames.append(observation)
        return self.observation(), reward, done, info

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        [self.frames.append(observation) for _ in range(self.num_stack)]
        return self.observation()

In [35]:
import gym
from tqdm import tqdm
import numpy as np

import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy

# Example for continuous actions
# env_id = "LunarLanderContinuous-v2"

# Example for discrete actions
#env_id = "CartPole-v1"

#env = gym.make(env_id)
env = Template_Gym()
#env = FrameStack(env, 4)
#env = Controller_Gym(env)
#env = DummyVecEnv([lambda: env])
#env = VecFrameStack(env, n_stack=1)
"""## Train Expert Model

We create an expert RL agent and let it learn to solve a task by interacting with the evironment.

"""

#ppo_expert = PPO('CnnPolicy', env, verbose=1, create_eval_env=True)
#ppo_expert.learn(total_timesteps=3e4, eval_freq=10000)
#ppo_expert.save("ppo_expert")
expert = MouseLogger()
"""check the performance of the trained agent"""

#mean_reward, std_reward = evaluate_policy(expert, env, n_eval_episodes=10)

#print(f"Mean reward = {mean_reward} +/- {std_reward}")

"""## Create Student

We also create a student RL agent, which will later be trained with the expert dataset

"""

ppo_student = PPO('CnnPolicy', env, verbose=1)

# only valid for continuous actions
# sac_student = SAC('MlpPolicy', env_id, verbose=1, policy_kwargs=dict(net_arch=[64, 64]))

"""
We now let our expert interact with the environment (except we already have expert data) and store resultant expert observations and actions to build an expert dataset.
"""

num_interactions = int(500)

if isinstance(env.action_space, gym.spaces.Box):
  expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
  expert_actions = np.empty((num_interactions,) + (env.action_space.shape[0],))

else:
  expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
  expert_actions = np.empty((num_interactions,) + env.action_space.shape)

obs = env.reset()

for i in tqdm(range(num_interactions)):
    action, _ = expert.actions(obs)
    print(action)
    expert_observations[i] = obs
    expert_actions[i] = action
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()

np.savez_compressed(
    "expert_data",
    expert_actions=expert_actions,
    expert_observations=expert_observations,
)

"""

- To seamlessly use PyTorch in the training process, we subclass an `ExpertDataset` from PyTorch's base `Dataset`.
- Note that we initialize the dataset with the previously generated expert observations and actions.
- We further implement Python's `__getitem__` and `__len__` magic functions to allow PyTorch's dataset-handling to access arbitrary rows in the dataset and inform it about the length of the dataset.
- For more information about PyTorch's datasets, you can read: https://pytorch.org/docs/stable/data.html.
"""

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Pointer moved to (914, 566)
Pointer moved to (914, 566)
Pressed at (914, 566)
Pointer moved to (914, 566)
Released at (914, 566)
start


  0%|          | 0/500 [00:00<?, ?it/s]

0


  0%|          | 1/500 [00:00<02:11,  3.81it/s]

0


...

  6%|▌         | 28/500 [00:18<02:52,  2.74it/s]

0


  6%|▌         | 29/500 [00:19<05:10,  1.52it/s]

0


KeyboardInterrupt: 

In [32]:
from torch.utils.data.dataset import Dataset, random_split

class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions
        
    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

"""

We now instantiate the `ExpertDataSet` and split it into training and test datasets.
"""

expert_dataset = ExpertDataSet(expert_observations, expert_actions)

train_size = int(0.8 * len(expert_dataset))

test_size = len(expert_dataset) - train_size

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

print("test_expert_dataset: ", len(test_expert_dataset))
print("train_expert_dataset: ", len(train_expert_dataset))

"""

NOTE: The supervised learning section of this code is adapted from: https://github.com/pytorch/examples/blob/master/mnist/main.py
1. We extract the policy network of our RL student agent.
2. We load the (labeled) expert dataset containing expert observations as inputs and expert actions as targets.
3. We perform supervised learning, that is, we adjust the policy network's parameters such that given expert observations as inputs to the network, its outputs match the targets (expert actions).
By training the policy network in this way the corresponding RL student agent is taught to behave like the expert agent that was used to created the expert dataset (Behavior Cloning).
"""

def pretrain_agent(
    student,
    batch_size=64,
    epochs=1000,
    scheduler_gamma=0.7,
    learning_rate=1.0,
    log_interval=100,
    no_cuda=True,
    seed=1,
    test_batch_size=64,
):
    use_cuda = not no_cuda and th.cuda.is_available()
    th.manual_seed(seed)
    device = th.device("cuda" if use_cuda else "cpu")
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    if isinstance(env.action_space, gym.spaces.Box):
      criterion = nn.MSELoss()
    else:
      criterion = nn.CrossEntropyLoss()

    # Extract initial policy
    model = student.policy.to(device)

    def train(model, device, train_loader, optimizer):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            if isinstance(env.action_space, gym.spaces.Box):
              # A2C/PPO policy outputs actions, values, log_prob
              # SAC/TD3 policy outputs actions only
              if isinstance(student, (A2C, PPO)):
                action, _, _ = model(data)
              else:
                # SAC/TD3:
                action = model(data)
              action_prediction = action.double()
            else:
              # Retrieve the logits for A2C/PPO when using discrete actions
              latent_pi, _, _ = model._get_latent(data)
              logits = model.action_net(latent_pi)
              action_prediction = logits
              target = target.long()

            loss = criterion(action_prediction, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )
    def test(model, device, test_loader):
        model.eval()
        test_loss = 0
        with th.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)

                if isinstance(env.action_space, gym.spaces.Box):
                  # A2C/PPO policy outputs actions, values, log_prob
                  # SAC/TD3 policy outputs actions only
                  if isinstance(student, (A2C, PPO)):
                    action, _, _ = model(data)
                  else:
                    # SAC/TD3:
                    action = model(data)
                  action_prediction = action.double()
                else:
                  # Retrieve the logits for A2C/PPO when using discrete actions
                  latent_pi, _, _ = model._get_latent(data)
                  logits = model.action_net(latent_pi)
                  action_prediction = logits
                  target = target.long()

                test_loss = criterion(action_prediction, target)
        test_loss /= len(test_loader.dataset)
        print(f"Test set: Average loss: {test_loss:.4f}")

    # Here, we use PyTorch `DataLoader` to our load previously created `ExpertDataset` for training
    # and testing
    train_loader = th.utils.data.DataLoader(
        dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
    )
    test_loader = th.utils.data.DataLoader(
        dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
    )

    # Define an Optimizer and a learning rate schedule.
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

    # Now we are finally ready to train the policy model.
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer)
        test(model, device, test_loader)
        scheduler.step()

    # Implant the trained policy network back into the RL student agent
    ppo_student.policy = model

"""Evaluate the agent before pretraining, it should be random"""

mean_reward, std_reward = evaluate_policy(ppo_student, env, n_eval_episodes=10)

print(f"Mean reward = {mean_reward} +/- {std_reward}")

"""

Having defined the training procedure we can now run the pretraining!
"""

pretrain_agent(
    ppo_student,
    epochs=10000000,
    scheduler_gamma=0.7,
    learning_rate=1.0,
    log_interval=100,
    no_cuda=True,
    seed=1,
    batch_size=64,
    test_batch_size=1000,
)
ppo_student.save("a2c_student")

"""

Finally, let us test how well our RL agent student learned to mimic the behavior of the expert
"""

mean_reward, std_reward = evaluate_policy(ppo_student, env, n_eval_episodes=10)

print(f"Mean reward = {mean_reward} +/- {std_reward}")

test_expert_dataset:  100
train_expert_dataset:  400


/home/adaptationai/anaconda3/envs/Shaman-AI/lib/python3.6/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
[([[0, 0], [51, 0], [51, 17], [0, 17]], 'SCOR', 0.9772217869758606)]
start
Mean reward = 0.0 +/- 0.0


RuntimeError: Given groups=1, weight of size [32, 1, 8, 8], expected input[64, 84, 84, 1] to have 1 channels, but got 84 channels instead

In [323]:
from stable_baselines import ACKTR, PPO2, SAC
env = Template_Gym()
#env = Controller_Gym(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=1)

expert_agent = MouseLogger()
model = expert_agent.actions
expert_agent 
      # Train a DQN agent for 1e5 timesteps and generate 10 trajectories
      # data will be saved in a numpy archive named `expert_cartpole.npz`


generate_expert_traj(expert_agent.actions, 'dummy_expert_cartpole', env, n_episodes=10)

Images will be recorded to recorded_images/
Image shape: (84, 84, 1)
Pointer moved to (914, 566)
Pointer moved to (914, 566)
Pressed at (914, 566)
Pointer moved to (914, 566)
Released at (914, 566)
start


TypeError: actions() takes 1 positional argument but 2 were given

In [12]:
from stable_baselines3.common.cmd_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack ,DummyVecEnv, VecNormalize
from stable_baselines3 import A2C, PPO, DQN
import torch
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.callbacks import CheckpointCallback, EveryNTimesteps

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
env = Template_Gym()
#env = Controller_Gym(env)
env = DummyVecEnv([lambda: env])
#obs = env.reset()
# It will check your custom environment and output additional warnings if needed
#check_env(env)
#env = VecFrameStack(env,4)
#env = ScaledFloatFrame(env)
#env = MaxAndSkipEnv(env, skip=1)
#env = BufferWrapper(env, 2)
#env = DummyVecEnv(env)
#env2 = make_atari_env(env, n_envs=1, seed=0)
#Frame-stacking with 4 frames
#env = VecNormalize(env)
env = VecFrameStack(env, n_stack=4)

checkpoint_on_event = CheckpointCallback(save_freq=1, save_path='./logsnew/')
event_callback = EveryNTimesteps(n_steps=2500, callback=checkpoint_on_event)

model = DQN('CnnPolicy', env, verbose=1)
#model.load("dgn_flappy_new"+str(0))
for i in range(100):
    try:
        model.load("dgn_flappy_new"+str(i-1))
        model.load_replay_buffer("dgn_flappy_new_replay"+str(i-1))
    except:
        print("cnat load")
    model.learn(total_timesteps=2500000, callback=event_callback)
    model.save("dgn_flappy_new"+str(i))
    model.save_replay_buffer("dgn_flappy_new_replay"+str(i))
    

#obs = env.reset()
#while True:
    #action, _states = model.predict(obs)
    #print(model.predict(obs))
    #print(model.predict(obs[0]))
    #obs, rewards, dones, info = env.step(action)
    #env.render()

#model.save("ppo_flappy_3")


Using cuda device
Wrapping the env in a VecTransposeImage.
cnat load
start


/home/adaptationai/anaconda3/envs/Shaman-AI/lib/python3.6/site-packages/stable_baselines3/common/buffers.py:220: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 56.46GB > 20.06GB
  "This system does not have apparently enough memory to store the complete "


-0.9295
start
-0.9505
start
-0.9535
start
-0.9595
start
----------------------------------
| rollout/            |          |
|    exploration rate | 0.999    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4        |
|    time_elapsed     | 30       |
|    total timesteps  | 138      |
----------------------------------
-0.958
start
-0.9535
start
-0.9625
start
-0.964
start
----------------------------------
| rollout/            |          |
|    exploration rate | 0.999    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4        |
|    time_elapsed     | 55       |
|    total timesteps  | 246      |
----------------------------------
-0.955
start
-0.955
start
-0.952
start
-0.9565
start
----------------------------------
| rollout/            |          |
|    exploration rate | 0.999    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4       

KeyboardInterrupt: 

In [6]:
from stable_baselines3.common.cmd_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack ,DummyVecEnv, VecNormalize
from stable_baselines3 import A2C, PPO, DQN
import torch

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
env = Template_Gym()
#env = Controller_Gym(env)
env = DummyVecEnv([lambda: env])
#obs = env.reset()
# It will check your custom environment and output additional warnings if needed
#check_env(env)
#env = VecFrameStack(env,4)
#env = ScaledFloatFrame(env)
#env = MaxAndSkipEnv(env, skip=1)
#env = BufferWrapper(env, 2)
#env = DummyVecEnv(env)
#env2 = make_atari_env(env, n_envs=1, seed=0)
#Frame-stacking with 4 frames
#env = VecNormalize(env)
env = VecFrameStack(env, n_stack=4)


#model = PPO('CnnPolicy', env, verbose=1)
model.load("ppo_flappy_"+str(37))
for i in range(100):
    try:
        model.load("ppo_flappy_new"+str(i-1))
        #model.load_replay_buffer("ppo_flappy_new_replay"+str(i-1))
    except:
        print("cnat load")
    model.learn(total_timesteps=250000)
    model.save("ppo_flappy_new"+str(i))
    #model.save_replay_buffer("ppo_flappy_new_replay"+str(i))
    

#obs = env.reset()
#while True:
    #action, _states = model.predict(obs)
    #print(model.predict(obs))
    #print(model.predict(obs[0]))
    #obs, rewards, dones, info = env.step(action)
    #env.render()


cnat load
start
-0.9805
start
-0.982
start
-0.9805
start
-0.979
start
-0.9775
start
-0.9805
start
-0.9835
start
-0.9805
start
-0.979
start
-0.982
start
-0.9775
start
-0.973
start
-0.9805
start
-0.982
start
-0.979
start
-0.976
start
-0.979
start
-0.9775
start
-0.979
start
-0.982
start
-0.9805
start
-0.9805
start
-0.979
start
-0.982
start
-0.9805
start
-0.9805
start
-0.976
start
-0.982
start
-0.9805
start
-0.982
start
-0.982
start
-0.979
start
-0.9805
start
-0.9835
start
-0.979
start
-0.979
start
-0.9805
start
-0.973
start
-0.979
start
-0.9805
start
-0.979
start
-0.9835
start
-0.9805
start
-0.9775
start
-0.982
start
-0.9775
start
-0.982
start
-0.9805
start
-0.979
start
-0.982
start
-0.982
start
-0.982
start
-0.9835
start
-0.9835
start
-0.979
start
-0.979
start
-0.973
start
-0.979
start
-0.979
start
-0.9805
start
-0.982
start
-0.9805
start
-0.9805
start
-0.9805
start
-0.9805
start
-0.982
start
-0.973
start
-0.9805
start
-0.979
start
-0.982
start
-0.976
start
-0.9805
start
-0.979
start
-0.

KeyboardInterrupt: 

In [49]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.cmd_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack ,DummyVecEnv, VecNormalize
from stable_baselines3 import A2C, PPO, DQN
import torch

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
env = Template_Gym()
#env = Controller_Gym(env)
env = DummyVecEnv([lambda: env])
#obs = env.reset()
# It will check your custom environment and output additional warnings if needed
#check_env(env)
#env = VecFrameStack(env,4)
#env = ScaledFloatFrame(env)
#env = MaxAndSkipEnv(env, skip=1)
#env = BufferWrapper(env, 2)
#env = DummyVecEnv(env)
#env2 = make_atari_env(env, n_envs=1, seed=0)
#Frame-stacking with 4 frames
#env = VecNormalize(env)
env = VecFrameStack(env, n_stack=4)
model = DQN.load("./logs/rl_model_712500_steps")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=20, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

start
-0.973
start
-0.9445
start
-0.973
start
-0.976
start
-0.976
start
-0.9279999999999999
start
-0.976
start
-0.9129999999999999
start
-0.943
start
-0.979
start
-0.9775
start
-0.9595
start
-0.9715
start
-0.9129999999999999
start
-0.964
start
-0.9715
start
-0.973
start
-0.9595
start
-0.9715
start
-0.9595
start
mean_reward=-0.96 +/- 0.020404411465120505


In [26]:
!pip install split-folders tqdm

In [35]:
import splitfolders
splitfolders.ratio('home/adaptationai/Documents/git/adaptationio/Shaman-AI/Shaman-AI/images', output="/home/adaptationai/Documents/git/adaptationio/Shaman-AI/Shaman-AI/output", seed=1337, ratio=(.8, 0.2,0.0))





Copying files: 0 files [00:00, ? files/s]


In [50]:
import splitfolders  
# The path to the directory where the original dataset was uncompressed
input_folder = '/home/adaptationai/Documents/git/adaptationio/Shaman-AI/Shaman-AI/images'
# The directory where we will store our smaller dataset
output_folder = '/home/adaptationai/Documents/git/adaptationio/Shaman-AI/Shaman-AI/output'

splitfolders.ratio(input_folder, output_folder, seed=1337, ratio=(.8, .1, .1), group_prefix=None) 

Copying files: 447 files [00:00, 10062.22 files/s]


In [384]:
r= [(0,None), (1,None)]
#2 = (np.array(r), None)
r2 = (r[0], None)
r3 = (r[0], None)
print(r2)
print(r2[0])
print(r3)
#response = (r)
#print(response)
#print(response[0])

((0, None), None)
(0, None)
((0, None), None)


In [25]:
from stable_baselines3.common.cmd_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack ,DummyVecEnv
from stable_baselines3 import A2C, PPO, DQN
import torch
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_checker import check_env

env = Template_Gym()
env = DummyVecEnv([lambda: env])
#obs = env.reset()
# It will check your custom environment and output additional warnings if needed
#check_env(env)
env = VecFrameStack(env,4)
# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
#env = Template_Gym()
#env = MaxAndSkipEnv(env, skip=4)
#env = ProcessFrame84(env)
#env = ImageToPyTorch(env) 
#env = BufferWrapper(env, 4)
#obs = env.reset()
#obs = env.step(1)
#bs = env.step(4)
#obs = env.step(4)
#obs = env.step(4)
obs.shape

NameError: name 'obs' is not defined

In [69]:
from stable_baselines3.common.cmd_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C, PPO, DQN
import torch
# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
env = Template_Gym()
#env = make_atari_env(env, n_envs=1, seed=0)
#Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)

#model = PPO('CnnPolicy', env, verbose=1)
#model.learn(total_timesteps=250000)

obs = env.reset()
obs.shape

[([[12, 8], [124, 8], [124, 34], [12, 34]], 'RESTART', 0.9832305775554079)]
RESTART
start


(84, 84, 1)

In [28]:
from stable_baselines3.common.cmd_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C
import torch
# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
env = make_atari_env('PongNoFrameskip-v4', n_envs=1, seed=0)
# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)

model = A2C('CnnPolicy', env, verbose=1)
#model.learn(total_timesteps=25000)

obs = env.reset()
#while True:
    #action, _states = model.predict(obs)
    #obs, rewards, dones, info = env.step(action)
    #env.render()
obs.shape

Using cuda device
Wrapping the env in a VecTransposeImage.


(1, 84, 84, 4)

In [274]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

--2021-09-13 08:56:44--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: 'Roms.rar.1’

Roms.rar.1          100%[===================>]  10.61M   175KB/s    in 64s     

2021-09-13 08:57:49 (169 KB/s) - 'Roms.rar.1’ saved [11128004/11128004]

mkdir: cannot create directory ‘/content/ROM/’: No such file or directory
/bin/bash: unrar: command not found


In [ ]:
import numpy as np
#from utilities import DataGrabber
#import torch
import random
#from ..common import DataGrabber
class TetrisEnv():
    def __init__(self):
        self.love = 14
        self.actions = [1,2,3,4]
        self.state = None
        self.state_full = None
        self.state_current = None
        self.count = 0
        self.load = True
        self.eval = False
        self.live_state =[2]
        self.live = True
        self.player = Player(self.config)
        #self.numbergen = NumberGen()

    
    def make_episode(self):
        #NumberGen.to_array(10000000, True)
        #self.numbergen = NumberGen()
        #self.numbergen.to_array(10000000, True)
        #self.state_full = np.load('data/game10000000.npy')
        #self.state_full []
        print('make episode')

    def make_current_state(self, count):
        if self.live:
            self.state = self.live_state
            
        else:
            start = (0+count+self.rand)
            end = (1001+count+self.rand)
            #self.state = self.state_full[start:end]
            
        return self.state

    def get_state(self):
        self.state_current = self.state[-1]
        return self.state_current

    def step(self, action):
        self.count += 1
        self.player.action(self.state_current, action)
        self.make_current_state(self.count)
        state = self.state_maker()
        reward = self.reward(self.get_state(), self.bet_value)
        self.player.balance += reward
        done = self.done(self.count)
        if done:
            self.render()
        
        return state, reward, done


    def reset(self):
        self.player.balance = 0
        self.count = 0
        self.make_episode()
        #if self.eval:
            #self.rand = np.random.random_integers(len(self.state_full / 10 * 9), len(self.state_full))
        #else:
            #self.rand = np.random.random_integers(len(self.state_full / 10 * 9))
        self.state = self.make_current_state(self.count)
        #print(len(self.state))
        state = self.state_maker()
        return state

    def render(self):
        print(self.player.balance)


    def state_maker(self):
        #user = self.player.details(self.count)
        state_details = self.state_details(self.state)
        count = np.array([self.count])
        state = self.data_grabber.flatten(state_details, count)

        return state

    def reward(self, state):
        reward = 0
    

        return reward
    
    def done(self, count):
        if count == 10000:
            
            return True
        else:
            return False 

    def state_details(self, state):
        ind = Indicators()
        details = []
        mean = ind.mean(state)
        median = ind.mean(state)
        black, red = ind.bandr(state)
        details.append([mean, median, black, red])
        return details[0]
         

         
class Player():
    def __init__(self, config):
        self.config = config
    
    def action_user(self, m_price, pm_price):
        #print(len)
        #self.update(m_price)
        x = input('buy, sell, close, hold?:')
        x = str(x)
        if x == "buy":
            self.open_position_long(m_price, pm_price)
        elif  x == "sell":
            self.open_position_short(m_price, pm_price)
        elif x == "close":
            self.close_position(m_price, pm_price)
        elif x == "hold":
            self.hold_position(m_price, pm_price)
        else:
            self.hold_position(m_price, pm_price)

    def action(self, m_price, action, pm_price):
        #print(len)
        #self.update(m_price)
        x = action
        #x = self.normalize(x)
        #x = int(x)
        #if self.placement < -0.200 or self.placement > 0.200:
            #self.close_position(m_price, pm_price)
        if self.config.buy == True:
        
            if x == 2:
                self.open_position_long(m_price, pm_price, x)
            elif x == 1:
                self.close_position(m_price, pm_price, x)
            elif x == 0:
                self.open_position_short(m_price, pm_price, x)
        else:   
            if x == 0:
                self.open_position_short(m_price, pm_price, x)
            elif x == 1:
                self.close_position(m_price, pm_price, x)
            elif x == 2:
                self.open_position_long(m_price, pm_price, x)
        if x == 3:
            self.hold_position(m_price, pm_price)

class PlayerAI():
    def __init__(self, config):
        self.config = config
        

    def action_user(self, m_price, pm_price):
        #print(len)
        #self.update(m_price)
        x = input('buy, sell, close, hold?:')
        x = str(x)
        if x == "buy":
            self.open_position_long(m_price, pm_price)
        elif  x == "sell":
            self.open_position_short(m_price, pm_price)
        elif x == "close":
            self.close_position(m_price, pm_price)
        elif x == "hold":
            self.hold_position(m_price, pm_price)
        else:
            self.hold_position(m_price, pm_price)

    def action(self, m_price, action, pm_price):
        #print(len)
        #self.update(m_price)
        x = action
        #x = self.normalize(x)
        #x = int(x)
        #if self.placement < -0.200 or self.placement > 0.200:
            #self.close_position(m_price, pm_price)
        if self.config.buy == True:
        
            if x == 2:
                self.open_position_long(m_price, pm_price, x)
            elif x == 1:
                self.close_position(m_price, pm_price, x)
            elif x == 0:
                self.open_position_short(m_price, pm_price, x)
        else:   
            if x == 0:
                self.open_position_short(m_price, pm_price, x)
            elif x == 1:
                self.close_position(m_price, pm_price, x)
            elif x == 2:
                self.open_position_long(m_price, pm_price, x)
        if x == 3:
            self.hold_position(m_price, pm_price)

         





In [1]:
import numpy as np
import pyautogui
import imutils
import cv2
import pytesseract
from pynput import mouse
import time
import easyocr
#pytesseract.pytesseract.tesseract_cmd = r'C:\Users\adaptation\anaconda3\envs\Shaman-AI\Library\bin\tesseract.exe'

In [ ]:
while True:
    try:

        state= pyautogui.screenshot(region=(1025, 260, 100, 50))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        cv2.imshow("Stream", state)
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break      
    except KeyboardInterrupt:
        cv2.destroyAllWindows()
        break

In [2]:
class ImageProcess():
    def __init__(self):
        self.love = "ramona"
        self.reader = easyocr.Reader(['en'])

    def on_click(self, x, y, button, pressed):
        if button == mouse.Button.left:
            print('{} at {}'.format('Pressed Left Click' if pressed else 'Released Left Click', (x, y)))
        
            return False # Returning False if you need to stop the program when Left clicked.
        else:
            print('{} at {}'.format('Pressed Right Click' if pressed else 'Released Right Click', (x, y)))
            return False # Returning False if you need to stop the program when Left clicked.

    def get_position(self):
        print("Please select top corner")
        listener = mouse.Listener(on_click=self.on_click)
        listener.start()
        listener.join()
        X=pyautogui.position()
        self.topx = X[0]
        self.topy= X[1]
        time.sleep(1)
        print(self.topx)
        print(self.topy)
        print("Please select bottom corner")
        listener = mouse.Listener(on_click=self.on_click)
        listener.start()
        listener.join()
        B=pyautogui.position()
        self.bottomx = B[0]
        self.bottomy= B[1]
        print(self.bottomx)
        print(self.bottomy)
        return self.topx, self.topy, self.bottomx, self.bottomy

    def image_ocr(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        #state = cv2.cvtColor(np.array(state), cv2.COLOR_BGR2GRAY)
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        #cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(state)
        print(text)
        return text

    def easy_ocr(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)

        #state = 255 - cv2.threshold(state, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Blur and perform text extraction
        #state = cv2.GaussianBlur(state, (3,3), 0)
        #self.reader = easyocr.Reader(['en'])
        result = self.reader.readtext(state)
        return result

    

    def image_ocr_2(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy
        custom_config = r'--oem 3 --psm 10'
        image = pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
        gray= cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
        thresh = 255 - cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Blur and perform text extraction
        thresh = cv2.GaussianBlur(thresh, (3,3), 0)
        text = pytesseract.image_to_string(thresh, config=custom_config)
        print(text)
        return text

    def image_stream(self):
        topx = self.topx
        topy = self.topy
        bottomx = self.bottomx - topx
        bottomy =  self.bottomy - topy 
        while True:
            try:
                state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
                state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
                cv2.imshow("Stream", state)
                if cv2.waitKey(1)& 0xFF == ord('q'):
                    break      
            except KeyboardInterrupt:
                cv2.destroyAllWindows()
                break


In [19]:
image_process = ImageProcess()
image_process.get_position()
#image_process. image_ocr()
#image_process.image_stream()



CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Please select top corner
Pressed Left Click at (847, 543)
847
543
Please select bottom corner
Pressed Left Click at (983, 589)
983
589


(847, 543, 983, 589)

In [120]:
image_process.image_stream()

In [12]:
image_process.image_ocr_2()

(Gera)



'(Gera)\n\x0c'

In [15]:
image_process.easy_ocr()

[([[16, 10], [130, 10], [130, 38], [16, 38]], 'RESTART', 0.9978891767276541)]

In [ ]:
def get_position():
    print("Please select top corner")
    listener = mouse.Listener(on_click=on_click)
    listener.start()
    listener.join()
    X=pyautogui.position()
    topx = X[0]
    topy= X[1]
    time.sleep(1)
    print(topx)
    print(topy)
    print("Please select bottom corner")
    listener = mouse.Listener(on_click=on_click)
    listener.start()
    listener.join()
    B=pyautogui.position()
    bottomx = B[0]
    bottomy= B[1]
    print(bottomx)
    print(bottomy)
    return topx, topy, bottomx, bottomy

In [ ]:

def image_ocr(tx, ty, bx, by):
    topx = tx
    topy = ty
    bottomx = bx - topx
    bottomy =  by - topy
    state= pyautogui.screenshot(region=(int(topx), int(topy), int(bottomx), int(bottomy)))
    state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
    text = pytesseract.image_to_string(state)
    print(text)
    return text



In [ ]:
# break between test and live code

In [ ]:


def on_click(x, y, button, pressed):
    if button == mouse.Button.left:
        print('{} at {}'.format('Pressed Left Click' if pressed else 'Released Left Click', (x, y)))
        
        return False # Returning False if you need to stop the program when Left clicked.
    else:
        print('{} at {}'.format('Pressed Right Click' if pressed else 'Released Right Click', (x, y)))
        return False # Returning False if you need to stop the program when Left clicked.
print("Please select top corner")
listener = mouse.Listener(on_click=on_click)
listener.start()
listener.join()
X=pyautogui.position()
topx = X[0]
topy= X[1]
time.sleep(1)
print(topx)
print(topy)
print("Please select bottom corner")
listener = mouse.Listener(on_click=on_click)
listener.start()
listener.join()
B=pyautogui.position()
bottomx = B[0]
bottomy= B[1]
print(bottomx)
print(bottomy)


In [1]:
import numpy as np
import pyautogui
import imutils
import cv2
import pytesseract
from bs4 import BeautifulSoup
from selenium import webdriver
import time
%matplotlib auto

pytesseract.pytesseract.tesseract_cmd = r'C:\Users\adaptation\anaconda3\envs\Shaman-AI\Library\bin\tesseract.exe'

Using matplotlib backend: TkAgg


In [2]:
image = pyautogui.screenshot(region=(0,0, 290, 30))
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
#image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
# image = cv2.GaussianBlur(image, (5, 5), 0)
#image = cv2.Canny(image, 50, 200, 255)
#image = cv2.medianBlur(image,5)
#cv2.imwrite("i1n_memory_to_disk.png", image)
#image = cv2.imread("in_memory_to_disk.png")
#cv2.imshow('HSV image', image); cv2.waitKey(1); cv2.destroyAllWindows(); cv2.waitKey(1)
#pytesseract.image_to_string(image, lang='eng', \
#config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
#custom_config = r'--oem 1 --psm 1'
#custom_config = r'--oem 3 --psm 6'
#game = pytesseract.image_to_string(image, config=custom_config)
#game = game.replace('x', '')
#game = game.replace('%', '')
#games.append(float(game))
#count = count + 1
#print(game)
text = pytesseract.image_to_string(image)
print(text)

lamenimenemen Ca




In [ ]:
while True:
    try:

        state= pyautogui.screenshot(region=(538, 373, 800, 600))
        state = cv2.cvtColor(np.array(state), cv2.COLOR_RGB2BGR)
        cv2.imshow("Stream", state)
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break      
    except KeyboardInterrupt:
        cv2.destroyAllWindows()
        break